In [ ]:
import os
import sys
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from tenacity import retry, wait_exponential, stop_after_attempt, Retrying, retry_if_exception_type
import google.api_core.exceptions

# Load environment variables
load_dotenv()
PINE_API_KEY = os.environ.get("PINECONE_api_key")
GEMINI_API_KEY = os.environ.get("Gemini_api_key")
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

# Pinecone vector store setup
index_name = "therapybot"
# You might need to handle the case where the index doesn't exist
# For example, using a try-except block
try:
    docsearch = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    )
except Exception as e:
    print(f"Pinecone index '{index_name}' does not exist or an error occurred: {e}")
    print("Creating and populating the index...")
    # This part would require your data loading and splitting logic to be here
    loader = DirectoryLoader("C:\\Users\\Shounak Dutta\\Documents\\Sih'25 hearMeout\\hearMeout\\Data",
                             glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    docsearch = PineconeVectorStore.from_documents(
        documents=text_chunks,
        index_name=index_name,
        embedding=embeddings,
    )

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# Gemini LLM setup with retry logic
# We use a retry decorator to automatically handle API errors,
# specifically the 429 RateLimitError and 500 Internal Server Errors.
@retry(
    wait=wait_exponential(multiplier=1, min=4, max=10),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type(google.api_core.exceptions.ResourceExhausted)
)
def create_llm_with_retry():
    return ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0.4,
        max_tokens=200,
        google_api_key=GEMINI_API_KEY
    )

llm = create_llm_with_retry()

# Prompt setup
system_prompt = (
    "You are a highly experienced and qualified mental health counsellor for high school students. "
    "Use the retrieved context and the user’s prompt to answer, give advice, and suggest things to ease the problem of the user. "
    "Use an empathetic and warm tone to not excite any reaction but to support the user. "
    "Take a deep breath and process all the context, then give an answer which best just advises and suggests rather than straight up solving the problem. "
    "Ask questions and ask for more context if required. "
    "You are the single thing supporting and comforting the user in any given situation, so tread lightly so as to not get too personal yet act as a warm counsellor to take care of the mental health of the user. "
    "If you don’t have straight answers or if the answers are too complex, start by recommending to seek warmth and comfort from adults, teachers, counsellors, or booking an appointment with the student therapist. "
    "Give answers in simple wording, short and friendly manner. "
    "Use a suggestive style and avoid making it sound compulsory or like an order. "
    "Take time to give answers and don’t make up information entirely. Be kind and warm. "
    "Ensure all replies have complete sentences and a full-stop is present at the end of the last sentence. "
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

@retry(
    wait=wait_exponential(multiplier=1, min=4, max=10),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type(google.api_core.exceptions.ResourceExhausted)
)
def invoke_rag_chain(input_text):
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)
    return rag_chain.invoke({"input": input_text})

# Global counter for the safety message
safety_message_counter = 0

def handle_user_input(user_input):
    """
    Checks for safety keywords and provides a repeated, pre-written response if found.
    Then, it always invokes the regular RAG chain to provide a full response.
    """
    global safety_message_counter
    
    # A list of safety-critical keywords (case-insensitive)
    safety_keywords = [
        "suicide", "kill myself", "end my life", "want to die", 
        "self-harm", "cutting myself", "no way out", "can't go on", 
        "hopeless", "desperate", "giving up", "hurt myself", "harm myself"
    ]
    
    # Check if a new safety keyword is in the user's input
    if any(keyword in user_input.lower() for keyword in safety_keywords):
        safety_message_counter = 3
        print("Thank you for reaching out. It sounds like you are going through something very difficult. Please, reach out to a trusted adult, like a teacher, a school counselor, or a family member, right now. You are not alone, and there are people who can help you.")

    # If the safety counter is active, print the repeated message and decrement
    if safety_message_counter > 0:
        safety_message_counter -= 1
        print("Remember, your well-being is important. Please connect with an adult you trust or a professional counselor. They are there to help.")

    # Always proceed with the regular LLM process
    response = invoke_rag_chain({"input": user_input})
    if "answer" in response:
        print(response["answer"])
    else:
        print("I'm sorry, I couldn't find a helpful answer based on the context. Please try rephrasing your question.")


# response = invoke_rag_chain("I am so down right now. I dont want to live")
response = invoke_rag_chain("I dont feel good today because i dont have a girfriend")

if "answer" in response:
    print(response["answer"])
else:
    print("No answer found.")


Oh, honey, it sounds like you're in a really tough place right now. I'm so sorry you're feeling this way. It takes courage to share something like that, and I want you to know I'm here to listen.

When you say you don't want to live, what does that feel like for you? Is it a heavy sadness, a complete lack of hope, or something else? Sometimes just describing the feeling can help us understand it a little better.

It's important to remember that these feelings, as intense as they are, are temporary. Like a storm, they will eventually pass. "My anxiety/fear/sadness won’t kill me; it just doesn’t feel good right now.”

You don't have to carry this weight alone. Would you be open to reaching out to someone who can provide more support? Maybe a trusted adult, a school counselor, or a mental health professional? They can offer


In [3]:
response = invoke_rag_chain("I dont feel good today because i dont have a girfriend")

if "answer" in response:
    print(response["answer"])
else:
    print("No answer found.")

Hey there, it sounds like you're feeling down because you don't have a girlfriend right now. That's completely understandable. It's normal to want to share your life with someone special.

Instead of focusing on what you don't have, maybe we could explore some things you *do* have and enjoy? Sometimes, shifting our perspective can make a big difference.

Also, have you thought about what specifically makes you want a girlfriend right now? Is it companionship, feeling loved, or something else? Understanding what you're looking for can help you find it, whether it's in a romantic relationship or in other areas of your life.

Remember, you're valuable and worthy of love and happiness, whether you're in a relationship or not.


In [6]:
response = invoke_rag_chain("Ayush touches me inappropriately and I feel sad and gropped, ptsd")

if "answer" in response:
    print(response["answer"])
else:
    print("No answer found.")

Oh darling, I'm so sorry to hear that Ayush touched you inappropriately. It sounds like you're going through a really tough time, and it's completely understandable that you're feeling sad and violated. No one has the right to make you feel unsafe or uncomfortable.

It's brave of you to share this with me. Remember, you're not alone, and there are people who care about you and want to help.

Given that you're feeling symptoms of PTSD, it might be a good idea to reach out to a trusted adult, like a parent, teacher, or counselor at school. They can provide you with support and guidance, and help you figure out the best way to cope with what happened.

In the meantime, here are a few things you can try to help yourself feel a little better:

*   **Grounding techniques:** When you're feeling overwhelmed, try focusing on your senses. Notice five things you can see,
